# Tutorial 1 - WaveBot
The goal of this tutorial is to familiarize new users with how to set up and run optimization problems using WecOptTool. 
It uses a one-body WEC, the WaveBot, in one degree of freedom in regular waves. 
The goal is to **find the optimal PTO force time-series** that produce the most power subject to a maximum force the PTO can exert.

![WaveBot Photo](https://live.staticflickr.com/65535/51855905347_de87ccaaba_z.jpg)

This tutorial covers three examples of increasing complexity:
* [Example 1: Single-loop optimization for mechanical power](#Example-1:-Single-loop-optimization-for-mechanical-power)
* [Example 2: Single-loop optimization for electrical power](#Example-2:-Single-loop-optimization-for-electrical-power)
* [Example 3: Multi-loop optimization for electrical power](#Example-3:-Multi-loop-optimization-for-electrical-power)

We will start by loading the necessary modules:

* Import Autograd (wrapper on NumPy, required) for automatic differentiation
* Import other packages we will use in this tutorial 
* Import WecOptTool 

In [ ]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt

# wecopttool
import wecopttool as wot

## Example 1: Single-loop optimization for mechanical power
This example illustrates how to set up, run, and analyze a basic optimization problem within WecOptTool.

WecOptTool requires the following to be defined to successfully run its optimization routines:
- The WEC object, including all of its properties and constraints
- The wave spectrum
- The objective function

### WEC object
In this section we will create the `WEC` object, which contains all the information about the WEC and its dynamics. This constitutes the vast majority of the setup required to run WecOptTool.

Our `WEC` object requires information about the mesh, degrees of freedom, mass and hydrostatic properties, linear hydrodynamic coefficients (from a BEM solution), any additional dynamic forces (e.g. PTO force, mooring, non-linear hydrodynamics), and constraints (e.g. maximum PTO extension). 
In this case, the only additional force will be the PTO force and the only constraint will be a maximum PTO force of $2000 N$.

##### Mesh
First, we will create a surface mesh for the hull and store it using the `FloatingBody` object from Capytaine. The WaveBot mesh is pre-defined in the `wecopttool.geom` module, so we will call it directly from there. We will only model the heave degree of freedom in this case. Note that the Capytaine `from_meshio` method can also import from other file types (STL, VTK, MSH, etc.)

In [ ]:
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

Next we will add the mass and hydrostatic stiffness properties. 
If these values are known they can be added directly.
Here we will use the fact that the WaveBot is free floating and assume constant density to calculate these properties, which Capytaine can natively perform with the `FloatingBody` created above. For convenience, this functionality has been wrapped in `wecopttool.hydrostatics`.

In [ ]:
stiffness = wot.hydrostatics.stiffness_matrix(fb).values
mass = wot.hydrostatics.inertia_matrix(fb).values

At this point we can visualize the mesh for inspection.
Capytaine has built-in methods for visualizing meshes (`fb.show`, and `fb.show_matplotlib`). 
When running outside a Notebook, these are interactive.  
The included WaveBot example also has a method for plotting the cross-section of the device. 

In [ ]:
fb.show_matplotlib()
_ = wb.plot_cross_section(show=True)  # specific to WaveBot

##### BEM
With our Capytaine floating body created, we can now run the Boundary Element Method solver in Capytaine to get the hydrostatic and hydrodynamic coefficients of our WEC object.This is wrapped into the `wecopttool.run_bem` function.

We will analyze 50 frequencies with a spacing of 0.05 Hz. These frequencies will be used for the Fourier representation of both the wave and the desired PTO force in the pseudo-spectral problem. See the Theory section of the Documentation for more details on the pseudo-spectral problem formulation.

If you would like to save our BEM data to a NetCDF file for future use, see the `wecopttool.write_netcdf` function.

In [ ]:
f1 = 0.05
nfreq = 50
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

bem_data = wot.run_bem(fb, freq)

##### PTO
WecOptTool includes the `PTO` class to encompass all properties of the power take-off system of the WEC. Data wrapped into our `PTO` class will be used to help define our `WEC` object and optimization problem later.

To create an instance of the `PTO` class, we need:
- The kinematics matrix, which converts from the WEC degrees of freedom to the PTO degrees of freedom. The PTO extracts power directly from the WEC's heave in this case, so the kinematics matrix is simply the $1 \times 1$ identity matrix.
- The definition of the PTO controller. The `wecopttool.pto` submodule includes P, PI, and PID controller functions that can be provided to the `PTO` class and return the PTO force. However, we will be using an unstructured controller in this case, so we will set `None` for the controller.
- Any PTO impedance. We're only interested in mechanical power for this first problem, so we will leave this empty for now
- The power conversion efficiency (assumed 100% if `None`)
- The PTO system name, if desired

In [ ]:
name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
efficiency = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, efficiency, name)

Now let's define the PTO forcing on the WEC and the PTO constraints. For our optimization problem, the constraints must be in the correct format for `scipy.optimize.minimize()`. We will enforce the constraint at 4 times more points than the dynamics (see Theory for why this is helpful for the pseudo-spectral problem).

In [ ]:
# PTO dynamics forcing function
f_add = {'PTO': pto.force_on_wec}

# Constraint
f_max = 2000.0
nsubsteps = 4

def const_f_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    f = pto.force_on_wec(wec, x_wec, x_opt, nsubsteps)
    return f_max - np.abs(f.flatten())

ineq_cons = {'type': 'ineq',
             'fun': const_f_pto,
             }
constraints = [ineq_cons]

##### `WEC` creation
We are now ready to create the `WEC` object itself! Since we ran our BEM already, we can define the object using the `wecopttool.WEC.from_bem` function. If we saved our BEM data to a NetCDF file, we can also provide the path to that file instead of specifying the BEM `Dataset` directly.

In [ ]:
wec = wot.WEC.from_bem(bem_data,
                    inertia_matrix=mass,
                    hydrostatic_stiffness=stiffness,
                    constraints=constraints,
                    friction=None,
                    f_add=f_add,
                    )

### Waves
The wave environment must be specified as an `xarray.Dataset` containing two 2-dimensional `xarray.DataArray`: (1) the amplitude spectrum  magnitude ``S``
(m^2*s) and (2) the phase ``phase`` (rad). 
The two coordinates are the wave frequency ``omega`` (rad/s)  and the direction ``wave_direction`` (rad). 
The `wecopttool.waves` submodule contains functions for creating this `xarray.Dataset` for different types of wave environments. 

In this case we will use a regular wave with a frequency of 0.3 Hz and an amplitude of 0.0625 m. 
We will use the `wecopttool.waves.regular_wave` function. 

Now we may create a regular wave using the `wecopttool.waves` submodule. In this case, we will use a wave with a frequency of 0.3 Hz and an amplitude of 0.0625 m.

In [ ]:
amplitude = 0.0625  
wavefreq = 0.3
phase = 30
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

### Objective function
The objective function is the quantity (scalar) we want to optimize -- in this case, the average mechanical power. The objective function is itself a function of the optimization state, the size of which we need to properly bound our call to `scipy.optimize.minimize()`. The average mechanical power can be taken directly from the `PTO` object we created:

In [ ]:
obj_fun = pto.mechanical_average_power
nstate_opt = 2*nfreq+1

### Solve
We are now ready to solve the problem. WecOptTool uses `scipy.optimize.minimize` as its optimization driver, which is wrapped into `wecopttool.WEC.solve` for ease of use.

Note that the only required inputs for defining and solving the problem are: (1) the waves, (2) the objective function, and (3) the size of the optimization state. Optional inputs can be provided to control the optimization execution if desired (see `scipy.optimize.minimize` docs for details). `solve` returns the frequency and time domain response of the WEC, with the optimal value of our objective function returned in the third output, `results`.

Pay attention to the `Exit mode`: an exit mode of $0$ indicates a successful solution.

`results` also contains the raw optimized state (i.e. the optimal PTO force Fourier coefficients), which we can postprocess into quantities of interest using the `wecopttool.pto.post_process` function.

In [ ]:
wec_fdom, wec_tdom, results = wec.solve(
    waves, 
    obj_fun, 
    nstate_opt, 
    )
opt_mechanical_average_power = results.fun

pto_fdom, pto_tdom = pto.post_process(wec, results, waves)

### Analyzing results

In [ ]:
plt.figure()


## Example 2: Single-loop optimization for electrical power

## Example 3: Multi-loop optimization for electrical power